In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Task, Crew

In [3]:
import os
from utils import get_groc_api_key

groc_api_key = get_groc_api_key()
#os.environ["GROQ_API_KEY"] = groc_api_key
os.environ["GROQ_API_KEY"] = 'deepseek-r1-distill-llama-70b'

In [4]:
# Create the LLM configuration
llm_config = {
    "api_key": groc_api_key,
    "model": "deepseek-r1-distill-llama-70b",
    "temperature": 0.7,
    "max_tokens": 500
}


In [5]:
researcher = Agent(
    role="Digital Marketing Researcher",
    goal="Gather accurate, relevant, and engaging information on {topic} to support the creation of a high-quality blog post.",
    backstory="You are an expert in digital marketing research, specializing in identifying trends, data, and insights that resonate with the target audience. "
              "Your role is to collect information that is factual, up-to-date, and valuable to the audience. "
              "Your research will serve as the foundation for the Content Writer to create an engaging and informative blog post.",
    instructions=[
        "1. Identify the target audience and their preferences for the topic: {topic}.",
        "2. Conduct thorough research using credible sources such as industry reports, case studies, and reputable websites.",
        "3. Gather data, statistics, and trends that support the topic and provide value to the audience.",
        "4. Organize the research into clear sections (e.g., introduction, key points, examples, and conclusions).",
        "5. Ensure all information is accurate, up-to-date, and properly cited.",
        "6. Provide a summary of the research findings to the Content Writer for further development."],
    allow_delegation=False,
	verbose=True
)

In [6]:
writer = Agent(
    role="Content Writer",
    goal="Write a compelling, engaging, and well-structured blog post on {topic} based on the research provided.",
    backstory="You are a skilled content writer with expertise in crafting blog posts that resonate with the target audience. "
              "Your role is to transform the research findings into a clear, engaging, and informative article that aligns with the company's tone and style.",
    instructions=[
        "1. Review the research provided by the Researcher and identify the key points to include in the blog post.",
        "2. Write a captivating headline and introduction to grab the reader's attention.",
        "3. Structure the blog post into sections (e.g., introduction, main body, conclusion) for easy readability.",
        "4. Use clear, concise, and engaging language to communicate the information effectively.",
        "5. Incorporate relevant examples, statistics, and quotes from the research to support your points.",
        "6. Ensure the tone and style align with the company's brand guidelines.",
        "7. Optimize the content for SEO by including relevant keywords and meta descriptions.",
        "8. Submit the draft to the Editor for review and refinement."
    ],
    allow_delegation=False,
    verbose=True
)

In [7]:
editor = Agent(
    role="Content Editor",
    goal="Review and refine the blog post to ensure it is polished, error-free, appropiate for the company's policy and ready for publication.",
    backstory="You are an experienced content editor with a keen eye for detail and a deep understanding of the company's brand voice. "
              "Your role is to ensure the blog post is clear campared to the {topic}, engaging, and free of errors before it is published.",
    instructions=[
        "1. Review the blog post draft for grammar, spelling, and punctuation errors.",
        "2. Ensure the content is clear, concise, and easy to understand.",
        "3. Verify that the tone and style align with the company's brand guidelines.",
        "4. Check that the blog post is well-structured and flows logically.",
        "5. Confirm that all facts, statistics, and quotes are accurate and properly cited.",
        "6. Provide constructive feedback to the Content Writer if revisions are needed.",
        "7. Finalize the blog post and prepare it for publication."
    ],
    allow_delegation=False,
    verbose=True
)

# Tools

In [8]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

c:\Users\DilukshaPereraBISTEC\OneDrive - BISTEC Global\Documents\MyProjects\LearnA_ws\learn\lib\site-packages\pydantic\_internal\_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
c:\Users\DilukshaPereraBISTEC\OneDrive - BISTEC Global\Documents\MyProjects\LearnA_ws\learn\lib\site-packages\pydantic\_internal\_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
c:\Users\DilukshaPereraBISTEC\OneDrive - BISTEC Global\Documents\MyProjects\LearnA_ws\learn\lib\site-packages\crewai_tools\tools\scrapegra

# Tasks

In [9]:
plan = Task(
    description=(
        "1. Analyze the topic: {topic} and identify its relevance to the target audience.\n"
        "2. Research the latest trends, statistics, and news related to {topic}.\n"
        "3. Identify key players, influencers, or brands in the industry related to {topic}.\n"
        "4. Gather credible sources such as industry reports, case studies, and reputable websites.\n"
        "5. Highlight pain points, challenges, and opportunities related to {topic}.\n"
        "6. Organize the research into clear sections (e.g., trends, data, examples, and sources)."
    ),
    expected_output=(
        "A detailed research document containing:\n"
        "- Key trends and statistics related to {topic}.\n"
        "- Credible sources and references.\n"
        "- Insights into the target audience's pain points and interests.\n"
        "- A structured outline of the research findings."
    ),
    tools = [search_tool, scrape_tool],
    agent=researcher,
)

In [10]:
audience_analysis = Task(
    description=(
        "1. Define the target audience for the blog post on {topic}.\n"
        "2. Identify the audience's demographics, interests, and preferences.\n"
        "3. Analyze the audience's pain points and how {topic} can address them.\n"
        "4. Determine the tone and style that will resonate with the audience.\n"
        "5. Provide recommendations for engaging the audience effectively."
    ),
    expected_output=(
        "An audience analysis report containing:\n"
        "- Demographics and psychographics of the target audience.\n"
        "- Pain points and interests related to {topic}.\n"
        "- Recommendations for tone, style, and engagement strategies."
    ),
    tools = [search_tool, scrape_tool],
    agent=researcher,
)

In [11]:
# writer task

write = Task(
    description=(
        "1. Use the research document and audience analysis to create a blog draft.\n"
        "2. Write a compelling headline and introduction to grab the reader's attention.\n"
        "3. Structure the blog post into sections (e.g., introduction, main body, conclusion).\n"
        "4. Incorporate key points, examples, and statistics from the research.\n"
        "5. Use a tone and style that aligns with the target audience and brand guidelines.\n"
        "6. Include a clear call-to-action (CTA) to engage the audience.\n"
        "7. Optimize the content for SEO by including relevant keywords and meta descriptions."
    ),
    expected_output=(
        "A complete blog draft containing:\n"
        "- A captivating headline and introduction.\n"
        "- Well-structured sections with clear key points.\n"
        "- Examples, statistics, and quotes from the research.\n"
        "- A strong call-to-action (CTA).\n"
        "- SEO-optimized content with keywords and meta descriptions."
    ),
    tools = [search_tool, scrape_tool],
    agent=writer,
)

In [12]:
revise_blog = Task(
    description=(
        "1. Review the Editor's feedback on the blog draft.\n"
        "2. Make necessary revisions to improve clarity, tone, and structure.\n"
        "3. Ensure all facts, statistics, and quotes are accurate and properly cited.\n"
        "4. Double-check SEO optimization, including keywords and meta descriptions.\n"
        "5. Submit the revised draft to the Editor for final approval."
    ),
    expected_output=(
        "A revised blog draft that incorporates all feedback and is ready for final review."
    ),
    tools = [search_tool, scrape_tool],
    agent=writer,
)

In [13]:
# editot task

edit = Task(
    description=(
        "1. Review the blog draft for grammar, spelling, and punctuation errors.\n"
        "2. Ensure the content is clear, concise, and easy to understand.\n"
        "3. Verify that the tone and style align with the company's brand guidelines.\n"
        "4. Check that the blog post is well-structured and flows logically.\n"
        "5. Confirm that all facts, statistics, and quotes are accurate and properly cited.\n"
        "6. Provide constructive feedback to the Writer for revisions if needed."
    ),
    expected_output=(
        "A reviewed blog draft with:\n"
        "- Corrections for grammar, spelling, and punctuation.\n"
        "- Suggestions for improving clarity, tone, and structure.\n"
        "- Confirmation of factual accuracy and proper citations.\n"
        "- Feedback for the Writer if revisions are required."
    ),
    tools = [search_tool, scrape_tool],
    agent=editor,
)

In [14]:
finalize_blog = Task(
    description=(
        "1. Review the revised blog draft from the Writer.\n"
        "2. Ensure all feedback has been incorporated and the content is error-free.\n"
        "3. Confirm that the blog post is optimized for SEO and aligns with brand guidelines.\n"
        "4. Prepare the blog post for publication by formatting it appropriately.\n"
        "5. Submit the finalized blog post for publishing."
    ),
    expected_output=(
        "A finalized blog post that is:\n"
        "- Error-free and polished.\n"
        "- SEO-optimized and aligned with brand guidelines.\n"
        "- Ready for publication."
    ),
    tools = [search_tool, scrape_tool],
    agent=editor,
)

# Crew

In [15]:
crew = Crew(
    agents=[researcher, writer, editor],
    tasks=[plan, audience_analysis, write, revise_blog, edit, finalize_blog],  # tasks oprates swquentially
    verbose=True,  # should be boolean
    memory = True
)

ValidationError: 1 validation error for Crew
  Value error, Please provide an OpenAI API key. You can get one at https://platform.openai.com/account/api-keys [type=value_error, input_value={'agents': [Agent(role=Di...': True, 'memory': True}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error

# Run Crew

In [ ]:
from IPython.display import Markdown
topic = "Write a blog post about How Microsoft Learn Student Ambassadors in Sri lanka"
result = crew.kickoff(inputs={"topic": topic})
Markdown(result.raw)

# Agent: Digital Marketing Researcher
## Task: 1. Analyze the topic: Write a blog post about How Microsoft Learn Student Ambassadors in Sri lanka and identify its relevance to the target audience.
2. Research the latest trends, statistics, and news related to Write a blog post about How Microsoft Learn Student Ambassadors in Sri lanka.
3. Identify key players, influencers, or brands in the industry related to Write a blog post about How Microsoft Learn Student Ambassadors in Sri lanka.
4. Gather credible sources such as industry reports, case studies, and reputable websites.
5. Highlight pain points, challenges, and opportunities related to Write a blog post about How Microsoft Learn Student Ambassadors in Sri lanka.
6. Organize the research into clear sections (e.g., trends, data, examples, and sources).


# Agent: Digital Marketing Researcher
## Thought: I need to gather recent and relevant information regarding the Microsoft Learn Student Ambassadors program in Sri Lanka to create a

```markdown
# Empowering the Next Generation: The Impact of Microsoft Learn Student Ambassadors in Sri Lanka

## Introduction
In a rapidly evolving digital landscape, the demand for skilled professionals in technology is at an all-time high. In Sri Lanka, the Microsoft Learn Student Ambassadors program is playing a transformative role in nurturing the next generation of tech innovators. This initiative not only equips university students with technical skills but also empowers them to take charge of their communities through leadership and collaboration.

## Growing Participation and Community Impact
### The Rise of Microsoft Learn Student Ambassadors
Since its inception, the Microsoft Learn Student Ambassadors program has seen a remarkable increase in participation from universities across Sri Lanka. Hundreds of enthusiastic students join hands to enhance their technical expertise while engaging with industry leaders. This is evidenced by impressive achievements of local teams, such as **Team Sipsara** and **The Green Tycoon**, who recently represented Sri Lanka in the World Finals of Microsoft Imagine Cup 2023. 

Team Sipsara developed a web-based application designed to measure and enhance the skills of children with Down Syndrome, while The Green Tycoon created "CarboMeter," a solution aimed at monitoring and achieving emission goals, contributing significantly to environmental sustainability. These students exemplify the program's aim to empower young minds to innovate solutions for pressing local and global issues.

### Engaging the Tech Community
Microsoft Learn Student Ambassadors actively facilitate workshops and community events that bridge the gap between academia and the tech industry. Significant events like the **Microsoft Build After Party 2023** and **Microsoft Ignite After Party 2022** have become cornerstones for networking and skill development, drawing students and tech enthusiasts together to learn and share knowledge.

## Skill Development and Networking Opportunities
### Technical Proficiency
Participants in this program report significant improvements in their technical skills. Areas such as **cloud technology**, **data analysis**, and **AI** are just a few fields that students engage in. Access to exclusive resources and training from Microsoft provides ambassadors with a unique edge in their academic and professional endeavors.

Ambassador engagement with expert mentors from Microsoft allows students to gain insights and practical guidance, further enriching their learning experience.

### Building Relationships
One of the most valuable aspects of the Microsoft Learn Student Ambassadors program is the strengthened network it fosters among students, educators, and industry professionals. Ambassadors have unparalleled access to mentors within Microsoft, offering guidance that significantly influences their future career paths.

## Overcoming Challenges
### Addressing Limitations
Despite the program’s successes, challenges remain. Many students still face issues like limited access to resources and mentorship within traditional educational setups. However, the Microsoft Learn Student Ambassadors program addresses these pain points by providing a structured pathway for students to gain practical experience and connect with industry leaders.

### Opportunities for Expansion
The program emphasizes collaborations with local tech firms, enabling ambassadors to engage in internships and real-world projects that complement their education. This approach not only enhances their resumes but also promotes reciprocal growth between academia and industry.

## Conclusion
The Microsoft Learn Student Ambassadors program is a beacon of opportunity in Sri Lanka, empowering students with essential skills and experiences to lead and innovate within their communities. As these ambassadors cultivate their skills and passions, they contribute to a progressive tech culture anticipated to create a significant impact on the future of technology in Sri Lanka.

Are you a university student eager to make an impact? Consider joining the Microsoft Learn Student Ambassadors program to embark on an exciting journey of learning, innovation, and community engagement!

### Call to Action
For more information about the Microsoft Learn Student Ambassadors program and how to participate, visit the [official site](https://studentambassadors.microsoft.com/) or connect with local ambassadors via their [LinkedIn community](https://www.linkedin.com/company/mlsasrilanka/). Don’t miss this chance to elevate your tech journey!

---
**Meta Description:** Discover how the Microsoft Learn Student Ambassadors program in Sri Lanka empowers students with essential tech skills and opportunities. Join a thriving community of future innovators today!
```

In this finalized blog post, I made sure to reflect the latest insights about the Microsoft Learn Student Ambassadors in Sri Lanka, including specific achievements and the community impact they have created. The tone remains inspiring and approachable for the target audience, and SEO elements like relevant links and key terminology have been well-incorporated.

git init
git add README.md
git commit -m "first commit"
git branch -M main
git remote add origin __link__
git push -u origin main

In [ ]:
'''notion agent/Include/ 
function calling'''

'notion agent/Include/ \nfunction calling'